# Currency Converter 

## Creating tools
1. Tool for fetching real-time data from API
2. Tool for multiplying currency with given amount

In [115]:
from langchain_community.tools import tool
from langchain_core.tools import InjectedToolArg   # Prevents setting up of arguments by the model before calling the tools
from typing import Annotated
from dotenv import load_dotenv 
import requests
import os

load_dotenv()

EXCHANGE_RATE_API_KEY = os.environ.get("EXCHANGE_RATE_API_KEY")

In [116]:
@tool
def get_currency_factor(base_currency:str, target_currency:str) -> float:
    """
    This function fetches the currency conversion factor between the base currency and the target currency.    
    """

    url = f" https://v6.exchangerate-api.com/v6/{EXCHANGE_RATE_API_KEY}/pair/{base_currency}/{target_currency}"
    response = requests.get(url)
    return response.json()

@tool
def convert_currency(base_currency_amount: float, conversion_factor: Annotated[float, InjectedToolArg]) -> float:  
    """
    Given a currency conversion factor this function calculates the target currency value from given base currency amount. 
    """

    return base_currency_amount * conversion_factor

In [117]:
get_currency_factor.invoke({"base_currency": "USD", "target_currency": "INR"})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1749600001,
 'time_last_update_utc': 'Wed, 11 Jun 2025 00:00:01 +0000',
 'time_next_update_unix': 1749686401,
 'time_next_update_utc': 'Thu, 12 Jun 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.608}

## Binding Tools

In [118]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI() 

llm_with_tool = llm.bind_tools([get_currency_factor, convert_currency])

In [119]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content="What is the conversion rate of usd to inr, and based on that convert 400 usd to inr.")]

In [120]:
ai_message = llm_with_tool.invoke(messages)
messages.append(ai_message)
ai_message

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rvIy1SoYKhc0koWpoHIevlOy', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "INR"}', 'name': 'get_currency_factor'}, 'type': 'function'}, {'id': 'call_h8TwhVjK8azZfsHqgmelgItb', 'function': {'arguments': '{"base_currency_amount": 400}', 'name': 'convert_currency'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 122, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bh6VvEXs3VfytMzhZqmnSmzocp7QV', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--cf40560d-8139-4adb-9764-bd2b2595fa3a-0', tool_calls=[{'name': 'get_currency_factor', 'args': {'b

In [121]:
ai_message.tool_calls

[{'name': 'get_currency_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_rvIy1SoYKhc0koWpoHIevlOy',
  'type': 'tool_call'},
 {'name': 'convert_currency',
  'args': {'base_currency_amount': 400},
  'id': 'call_h8TwhVjK8azZfsHqgmelgItb',
  'type': 'tool_call'}]

- Because of Annoted and InjectedToolArg a hallucinated value is not filled in the args of convert_currency 

## Tool Execution 

1. Execute the first tool to get the conversion rate.
2. Execute the second tool to obtain the amount in targeted currency    

In [122]:
import json 
for tool_call in ai_message.tool_calls: 
    if tool_call['name'] == 'get_currency_factor': 
        tool_message1 = get_currency_factor.invoke(tool_call)
        # fetch conversion rate 
        conversion_rate = json.loads(tool_message1.content)['conversion_rate']
        # append tool_message1 to message list
        messages.append(tool_message1)
    if tool_call['name'] == 'convert_currency':
        # fetch the current arg
        tool_call['args']['conversion_factor'] = conversion_rate 
        tool_message2 = convert_currency.invoke(tool_call)
        messages.append(tool_message2)

In [123]:
messages

[HumanMessage(content='What is the conversion rate of usd to inr, and based on that convert 400 usd to inr.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rvIy1SoYKhc0koWpoHIevlOy', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "INR"}', 'name': 'get_currency_factor'}, 'type': 'function'}, {'id': 'call_h8TwhVjK8azZfsHqgmelgItb', 'function': {'arguments': '{"base_currency_amount": 400}', 'name': 'convert_currency'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 122, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bh6VvEXs3VfytMzhZqmnSmzocp7QV', 'service_tier': 'default',

In [125]:
answer = llm_with_tool.invoke(messages)
print(answer.content)

The conversion rate of USD to INR is 85.608. 
Based on this rate, 400 USD is equal to 34,243.20 INR.


### This project was not an AI Agent because it was not autonomous, we had to call the tool by ourselves. For an AI Agent these tool callings are automatic

#### How would've an AI Agent workflow looked like 

1. User says: "Convert 10 USD to INR"
2. LLM thinks: "I don't know the rate. First, let me call get_conversion_factor"
3. Tool result comes : 85.3415
4. LLM tools at result THINK again: "Now I know the rate, next I should call convert_currency with 10 and 85.3415"
5. Tool result comes: 853.415 INR
6. LLM summarizes: "10 USD is 853.415 INR at current rate." 